## Download Data

In [ ]:
! wget https://storage.yandexcloud.net/yandex-research/shifts/weather/canonical-partitioned-dataset.tar

--2023-01-26 18:54:51--  https://storage.yandexcloud.net/yandex-research/shifts/weather/canonical-partitioned-dataset.tar
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7472220160 (7.0G) [application/x-tar]
Saving to: ‘canonical-partitioned-dataset.tar’

canonical-partition 100%[===================>]   6.96G  20.3MB/s    in 5m 59s  

2023-01-26 19:00:51 (19.9 MB/s) - ‘canonical-partitioned-dataset.tar’ saved [7472220160/7472220160]



In [ ]:
!tar -xvf canonical-partitioned-dataset.tar

In [ ]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 20.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.2/15.2 MB 121.5 MB/s eta 0:00:0000:0100:01


## Load Data into pd

In [ ]:
import pandas as pd

In [ ]:
train = pd.read_csv('../canonical-paritioned-dataset/shifts_canonical_train.csv')
dev = pd.read_csv('../canonical-paritioned-dataset/shifts_canonical_dev_out.csv')
test = pd.read_csv('../canonical-paritioned-dataset/shifts_canonical_eval_out.csv')

In [ ]:
dev = pd.read_csv('../canonical-paritioned-dataset/shifts_canonical_dev_out.csv')
test = pd.read_csv('../canonical-paritioned-dataset/shifts_canonical_eval_out.csv')

In [ ]:
!rm -r ../canonical-paritioned-dataset

In [ ]:
train['fact_cwsm_class'].value_counts()

0.0     1185355
10.0     976063
20.0     731382
21.0     113505
11.0      94736
23.0      18274
13.0       7634
22.0       1795
12.0        848
Name: fact_cwsm_class, dtype: int64

In [ ]:
def filter_dataset(df: pd.DataFrame):
    return df[(df['fact_cwsm_class'] == 0.)| (df['fact_cwsm_class'] == 10.)]

In [ ]:
train = filter_dataset(train)
dev = filter_dataset(dev)
test = filter_dataset(test)

In [ ]:
X_train = train.iloc[:,6:]
y_train = train['fact_cwsm_class']
X_dev = dev.iloc[:,6:]
y_dev = dev['fact_cwsm_class']
X_test = test.iloc[:,6:]
y_test = test['fact_cwsm_class']

In [ ]:
! mkdir assets/data/weather

In [ ]:
pd.concat([X_train, y_train], axis=1).to_csv('assets/data/weather/train.csv', index=False)
pd.concat([X_dev, y_dev], axis=1).to_csv('assets/data/weather/dev.csv', index=False)
pd.concat([X_test, y_test], axis=1).to_csv('assets/data/weather/test.csv', index=False)

In [ ]:
!ls -alh assets/data/weather

total 4.0G
drwxr-xr-x  2 birk birk 4.0K Jan 26 19:56 .
drwxr-xr-x 11 birk birk 4.0K Jan 26 19:49 ..
-rw-r--r--  1 birk birk  54M Jan 26 19:56 dev.csv
-rw-r--r--  1 birk birk 543M Jan 26 19:57 test.csv
-rw-r--r--  1 birk birk 3.4G Jan 26 19:56 train.csv


## Baseline Prediction

In [ ]:
import catboost

In [ ]:
model = catboost.CatBoostClassifier(iterations=1000, learning_rate=0.03, depth=2, loss_function='Logloss')
model.fit(X_train, y_train, verbose=100)

0:	learn: 0.6862782	total: 76.2ms	remaining: 1m 16s
100:	learn: 0.5235198	total: 7.31s	remaining: 1m 5s
200:	learn: 0.5069747	total: 14.3s	remaining: 56.8s
300:	learn: 0.5002165	total: 21.1s	remaining: 49.1s
400:	learn: 0.4964211	total: 27.7s	remaining: 41.4s
500:	learn: 0.4937951	total: 34.4s	remaining: 34.2s
600:	learn: 0.4917484	total: 41s	remaining: 27.2s
700:	learn: 0.4900046	total: 47.7s	remaining: 20.3s
800:	learn: 0.4886092	total: 54.3s	remaining: 13.5s
900:	learn: 0.4872878	total: 1m	remaining: 6.7s
999:	learn: 0.4861102	total: 1m 7s	remaining: 0us


<catboost.core.CatBoostClassifier>

In [ ]:
model.score(X_train, y_train)

0.7695957006002541

In [ ]:
model.score(X_dev, y_dev)

0.688774465346245

In [ ]:
model.score(X_test, y_test)

0.674320240014876